In [1]:
import pandas as pd
import numpy as np
import re
from zipfile import ZipFile

In [2]:
years=range(2000,2021)
All_DF=np.zeros([0,20])
for year in years:
    print(year)
    with ZipFile('Weather History/'+str(year)+'.zip') as myzip:
        path_documents=[name for name in myzip.namelist() if 'INMET_SE_SP' in name]
        for path in path_documents:
            base_name_match=re.search('SE_(\w\w)_([AC]\d+)_(.*?)_',path)
            with myzip.open(path) as myfile:
                DF=pd.read_csv(myfile,delimiter=';',skiprows=8,encoding='latin')
                DF['state']=base_name_match[1]
                DF['station']=base_name_match[2]+' '+base_name_match[2]
                if All_DF.shape[0]==0:
                    All_DF=DF
                else:
                    All_DF=pd.concat([All_DF,DF],ignore_index=True)
All_DF.head()

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011


KeyboardInterrupt: 

In [ ]:
All_DF[['TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)','TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)','TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)']]

In [ ]:
#We see that a couple of important columns got duplicated, so we fix that
All_DF.loc[All_DF['DATA (YYYY-MM-DD)'].isna(),'DATA (YYYY-MM-DD)']=All_DF['Data'][All_DF['DATA (YYYY-MM-DD)'].isna()]
All_DF.loc[All_DF['HORA (UTC)'].isna(),'HORA (UTC)']=All_DF['Hora UTC'][All_DF['HORA (UTC)'].isna()]

In [ ]:
#We keep only the desired variables and change their names
All_DF=All_DF[['DATA (YYYY-MM-DD)','HORA (UTC)','station',
               'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
               'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
               'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)']]
Column_Names={'DATA (YYYY-MM-DD)':'date',
              'HORA (UTC)':'hour',
               'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':'rain_mm',
               'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)':'temp_max',
               'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)':'temp_min'}
All_DF.rename(Column_Names,axis=1,inplace=True)

In [ ]:
All_DF.dtypes

In [ ]:
#Fix Numbers
def fix_numbers(x): 
    x=str(x); x=x.replace(',','.')
    try:
        x=float(x)
        if x<-100:
            return np.nan
        else:
            return x
    except:
        return np.nan
All_DF['rain_mm']=All_DF['rain_mm'].apply(fix_numbers)
All_DF['temp_max']=All_DF['temp_max'].apply(fix_numbers)
All_DF['temp_min']=All_DF['temp_min'].apply(fix_numbers)

In [ ]:
#Fix Hour
def fix_hour(x):
    x=str(x)
    x=x.replace(':','')
    x=x.split(' ')[0]
    x=int(x)/100
    return x
All_DF['hour']=All_DF['hour'].apply(fix_hour)

In [ ]:
#Fix date
def fix_date(x):
    x=str(x)
    x=x.replace('/','-')
    return x
All_DF['date']=All_DF['date'].apply(fix_date)
All_DF['date']=pd.to_datetime(All_DF['date'])

In [ ]:
All_DF.describe()

In [ ]:
All_DF.head()

In [ ]:
All_DF.dtypes

In [ ]:
All_DF.groupby('station').agg(lambda x: sum(x.isnull()))

In [ ]:
All_DF=All_DF[All_DF.station!='C891 CRIOSFERA']

In [ ]:
All_DF.to_csv('ima2021_project/Data/historic_weather_all_SP_hourly.csv.zip',index=False)

In [ ]:
All_DF.head()

In [ ]:
def sum2(x):
    if all(x.isnull()):
        return np.nan
    else:
        return np.sum(x)
    
All_DF_daily=All_DF.groupby(['station','date']).agg({'rain_mm':sum2,'temp_max':'max','temp_min':'min'}).reset_index()

In [ ]:
All_DF_daily.reset_index().columns

In [ ]:
All_DF.columns

In [ ]:
All_DF_daily.to_csv('ima2021_project/Data/historic_weather_all_SP_daily.csv',index=False)

In [ ]:
#Count how many are completely null.
All_DF.groupby('date').sum().isnull().sum()

In [ ]:
All_DF[All_DF.rain_mm.isnull()].groupby(['station','date']).agg(lambda x: 1).groupby('station').sum()[['hour']].rename({'hour':'Days Missing Rain'},axis=1)

In [ ]:
Rain=All_DF[['date','hour','station','rain_mm']]
Rain=Rain.pivot(index=['date','hour'],columns='station',values='rain_mm')

In [ ]:
Rain=Rain.groupby('date').agg(lambda x: all(x.isnull()))

In [ ]:
Rain['How Many']=Rain[['A705 BAURU','A711 SAO CARLOS','A737 IBITINGA','A741 BARRA BONITA']].sum(axis=1)

In [ ]:
Missing_Rain=Rain.groupby('How Many').count()[['A705 BAURU']].reset_index()
Missing_Rain.columns=['How Many Stations','Days of Missing Rain Info']
Missing_Rain

In [ ]:
Rain=Rain.reset_index()
Rain.plot(x='date',y='How Many', kind='scatter')

In [ ]:
All_DF.dtypes